## Generating facies realizations with GeostatsPy

We generate here a set of facies realizations (sand/ shale) using the fantastic geostatistical library _geostatspy_ (https://github.com/GeostatsGuy/GeostatsPy) by Michael Pyrcz (University of Texas, Austin).

The example is modified from his notebook https://github.com/GeostatsGuy/PythonNumericalDemos/GeostatsPy_sisim.ipynb



# OFFLINE VERSION 

Version to generate multiple fields, no plot output

Note: can be converted to Python script using:

```$ jupyter nbconvert --to script ITgeo_generating_facies_fields_script.ipynb```


In [5]:
import os                                                 # to set current working directory 
import numpy as np                                        # arrays and matrix math
import pandas as pd                                       # DataFrames
import matplotlib.pyplot as plt                           # plotting
import geostatspy.GSLIB as GSLIB
import geostatspy.geostats as geostats



### Load Data

We are using a data set that is also provided by Michael Pyrcz on https://github.com/GeostatsGuy/GeoDataSets

In [6]:
df = pd.read_csv("sample_data_MV_biased.csv")             # read a .csv file in as a DataFrame
df.describe()                                             # summary statistics 
df = df.sample(50)                                        # extract 50 samples
df = df.reset_index()                                     # reset the record index 
df.head(n=10)

,index,Unnamed: 0,X,Y,Facies,Porosity,Perm,AI
0,205,338,80.0,339.0,1.0,0.097085,1.157147,5861.952750
1,236,387,640.0,929.0,1.0,0.157730,76.628367,4870.919942
2,5,9,200.0,900.0,1.0,0.129949,23.348473,4781.590782
3,278,443,720.0,899.0,1.0,0.148054,18.235939,4657.906573
4,337,531,830.0,559.0,0.0,0.105352,3.970239,4396.191300
5,102,170,50.0,809.0,1.0,0.123712,2.874881,4555.093964
6,357,562,930.0,909.0,0.0,0.100640,0.605332,4061.720049
7,216,353,880.0,769.0,0.0,0.111088,0.463731,3763.963644
8,146,236,290.0,909.0,1.0,0.106622,18.285043,4794.296936
9,169,273,270.0,219.0,0.0,0.062169,0.408779,6244.965113


### Generating Realizations

In [7]:
# Sequential Indicator Simulation with Simple Kriging Multiple Realizations 
nx = 50; ny = 50; xsiz = 20.0; ysiz = 20.0; xmn = 10.0; ymn = 10.0; nxdis = 1; nydis = 1
ndmin = 0; ndmax = 10; nodmax = 10; radius = 400; skmean = 0
tmin = -999; tmax = 999
dummy_trend = np.zeros((11,11))            # the current version requires trend input - if wrong size it is ignored 

ncut = 2                                   # number of facies
thresh = [0,1]                             # the facies categories (use consisten order)
gcdf = [0.4,0.6]                           # the global proportions of the categories
varios = []                                # the variogram list
varios.append(GSLIB.make_variogram(nug=0.0,nst=1,it1=1,cc1=1.0,azi1=0,hmaj1=100,hmin1=100)) # shale indicator variogram
# varios.append(GSLIB.make_variogram(nug=0.0,nst=1,it1=1,cc1=1.0,azi1=0,hmaj1=100,hmin1=100)) # sand indicator variogram
varios.append(GSLIB.make_variogram(nug=0.0,nst=1,it1=1,cc1=1.0,azi1=0,hmaj1=100,hmin1=100)) # sand indicator variogram


n_realizations = 10
np.random.seed(seed = 12345)
seeds = np.random.choice(10000, n_realizations, replace=False)
seeds

array([7867, 1402, 8606, 8885, 6494, 9628, 7459, 3964, 7000, 4470])

In [8]:
realizations = np.empty(shape=(n_realizations, nx, ny))

for i in range(n_realizations):
    realizations[i,:,:] = geostats.sisim(df,'X','Y','Facies',ivtype=0,koption=0,ncut=2,thresh=thresh,gcdf=gcdf,trend=dummy_trend,
                   tmin=tmin,tmax=tmax,zmin=0.0,zmax=1.0,ltail=1,ltpar=1,middle=1,mpar=0,utail=1,utpar=2,
                   nx=nx,xmn=xmn,xsiz=xsiz,ny=ny,ymn=ymn,ysiz=ysiz,seed=seeds[i],
                   ndmin=ndmin,ndmax=ndmax,nodmax=nodmax,mults=1,nmult=3,noct=-1,radius=radius,ktype=0,vario=varios)

# sim_ik2 = geostats.sisim(df,'X','Y','Facies',ivtype=0,koption=0,ncut=2,thresh=thresh,gcdf=gcdf,trend=dummy_trend,
#                tmin=tmin,tmax=tmax,zmin=0.0,zmax=1.0,ltail=1,ltpar=1,middle=1,mpar=0,utail=1,utpar=2,
#                nx=nx,xmn=xmn,xsiz=xsiz,ny=ny,ymn=ymn,ysiz=ysiz,seed = 73074,
#                ndmin=ndmin,ndmax=ndmax,nodmax=nodmax,mults=1,nmult=3,noct=-1,radius=radius,ktype=0,vario=varios)

# sim_ik3 = geostats.sisim(df,'X','Y','Facies',ivtype=0,koption=0,ncut=2,thresh=thresh,gcdf=gcdf,trend=dummy_trend,
#                tmin=tmin,tmax=tmax,zmin=0.0,zmax=1.0,ltail=1,ltpar=1,middle=1,mpar=0,utail=1,utpar=2,
#                nx=nx,xmn=xmn,xsiz=xsiz,ny=ny,ymn=ymn,ysiz=ysiz,seed = 73075,
#                ndmin=ndmin,ndmax=ndmax,nodmax=nodmax,mults=1,nmult=3,noct=-1,radius=radius,ktype=0,vario=varios)

Data for IK3D: Variable column Facies
  Number   = 50
Setting up rotation matrices for variogram and search
Working on a single realization, seed 7867
   currently on node 0
   currently on node 250
   currently on node 500
   currently on node 750
   currently on node 1000
   currently on node 1250
   currently on node 1500
   currently on node 1750
   currently on node 2000
   currently on node 2250
Data for IK3D: Variable column Facies
  Number   = 50
Setting up rotation matrices for variogram and search
Working on a single realization, seed 1402
   currently on node 0
   currently on node 250
   currently on node 500
   currently on node 750
   currently on node 1000
   currently on node 1250
   currently on node 1500
   currently on node 1750
   currently on node 2000
   currently on node 2250
Data for IK3D: Variable column Facies
  Number   = 50
Setting up rotation matrices for variogram and search
Working on a single realization, seed 8606
   currently on node 0
   currently on 

### Save realizations

Save realizations for further use later

In [11]:
import pickle

In [29]:
f = open("facies_realizations.pkl", mode='wb')
pickle.dump(realizations, f)
f.close()